In [1]:
import time
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Multiply
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve, auc

In [2]:
X_train = pd.read_csv('/content/X_train_final.csv').values
y_train = pd.read_csv('/content/y_train_final.csv').values
X_val = pd.read_csv('/content/X_val_processed.csv').values
y_val = pd.read_csv('/content/y_val_processed.csv').values
X_test = pd.read_csv('/content/X_test_processed.csv').values
y_test = pd.read_csv('/content/y_test_processed.csv').values

y_train = y_train.astype(np.float32)
y_val = y_val.astype(np.float32)
y_test = y_test.astype(np.float32)

X_train_combined = np.concatenate((X_train, X_val), axis=0)
y_train_combined = np.concatenate((y_train, y_val), axis=0)

input_dim = X_train.shape[1]

best_params = {
    'initial_dense_units': 32,
    'num_dense_layers_after_attention': 3,
    'dense_units_L1': 64,
    'dense_units_L2': 64,
    'dense_units_L3': 16,
    'attention_transform_units': 64,
    'learning_rate': 0.0073866678565409506,
    'batch_size': 32,
    'optimizer': 'rmsprop',
    'dropout_rate': 0.25,
}

In [3]:
class FeatureAttentionLayer(tf.keras.layers.Layer):
    def __init__(self, attention_transform_units=None, **kwargs):
        super(FeatureAttentionLayer, self).__init__(**kwargs)
        self.attention_transform_units = attention_transform_units

    def build(self, input_shape):
        if self.attention_transform_units is None:
            self.attention_transform_units = input_shape[-1]
        self.attention_transform_layer = Dense(self.attention_transform_units, activation='tanh', use_bias=True, name='attention_transform')
        self.attention_weights_layer = Dense(input_shape[-1], activation='softmax', use_bias=False, name='attention_weights')
        super(FeatureAttentionLayer, self).build(input_shape)

    def call(self, inputs):
        transformed_inputs = self.attention_transform_layer(inputs)
        attention_weights = self.attention_weights_layer(transformed_inputs)
        weighted_inputs = Multiply()([inputs, attention_weights])
        return weighted_inputs

    def get_config(self):
        config = super(FeatureAttentionLayer, self).get_config()
        config.update({'attention_transform_units': self.attention_transform_units})
        return config

In [4]:
def build_no_dropout_model(input_dim, params):
    tf.keras.backend.clear_session()

    initial_dense_units = params['initial_dense_units']
    num_dense_layers = params['num_dense_layers_after_attention']
    learning_rate = params['learning_rate']

    optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)

    inputs = Input(shape=(input_dim,), name='Input')
    x = Dense(initial_dense_units, activation='relu', name='initial_dense')(inputs)

    x = FeatureAttentionLayer(attention_transform_units=params['attention_transform_units'], name='feature_attention_layer')(x)

    dense_units_list = [params['dense_units_L1'], params['dense_units_L2'], params['dense_units_L3']]

    for i in range(num_dense_layers):
        x = Dense(dense_units_list[i], activation='relu', name=f'dense_L{i+1}')(x)

    outputs = Dense(1, activation='sigmoid', name='Output')(x)

    model = Model(inputs=inputs, outputs=outputs, name='MLP_No_Dropout')
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC()])

    return model

In [5]:
print("--- Building and Training Ablated Model: No Dropout ---")
no_dropout_model = build_no_dropout_model(input_dim, best_params)
no_dropout_model.summary()

--- Building and Training Ablated Model: No Dropout ---


Model: "MLP_No_Dropout"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Input (InputLayer)              │ (None, 12)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ initial_dense (Dense)           │ (None, 32)             │           416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ feature_attention_layer         │ (None, 32)             │         4,160 │
│ (FeatureAttentionLayer)         │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_L1 (Dense)                │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_L2 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_L3 (Dense)                │ (None, 16)             │         1,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Output (Dense)                  │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,905 (46.50 KB)

 Trainable params: 11,905 (46.50 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, mode='min', restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-6, mode='min')
]

In [7]:
no_dropout_model.fit(
    X_train_combined, y_train_combined,
    validation_split=0.1,
    epochs=200,
    batch_size=best_params['batch_size'],
    callbacks=callbacks,
    verbose=1
)

Epoch 1/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.6846 - auc: 0.7535 - loss: 0.5675 - val_accuracy: 0.8125 - val_auc: 0.8977 - val_loss: 0.3762 - learning_rate: 0.0074
Epoch 2/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8389 - auc: 0.9150 - loss: 0.3651 - val_accuracy: 0.8438 - val_auc: 0.9097 - val_loss: 0.3707 - learning_rate: 0.0074
Epoch 3/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8690 - auc: 0.9297 - loss: 0.3221 - val_accuracy: 0.8203 - val_auc: 0.9177 - val_loss: 0.3753 - learning_rate: 0.0074
Epoch 4/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8925 - auc: 0.9418 - loss: 0.2909 - val_accuracy: 0.8125 - val_auc: 0.9233 - val_loss: 0.4571 - learning_rate: 0.0074
Epoch 5/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8920 - auc: 0.9517 - loss: 0.2790 - val_accuracy: 0.8750 - val_auc: 0.9371 - val_loss: 0.2961 - learning_rate: 0.0074
Epoch 6/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9217 - auc: 0.9588 

In [8]:
test_loss, test_accuracy, test_auc = no_dropout_model.evaluate(X_test, y_test, verbose=0)
y_pred_proba = no_dropout_model.predict(X_test).ravel()
y_pred_binary = (y_pred_proba > 0.5).astype(int)
precision = precision_score(y_test, y_pred_binary)
recall = recall_score(y_test, y_pred_binary)
f1 = f1_score(y_test, y_pred_binary)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step


In [9]:
trainable_count = np.sum([K.count_params(w) for w in no_dropout_model.trainable_weights])
non_trainable_count = np.sum([K.count_params(w) for w in no_dropout_model.non_trainable_weights])
total_parameters = trainable_count + non_trainable_count

In [10]:
start_time = time.time()
no_dropout_model.predict(X_test, batch_size=best_params['batch_size'])
end_time = time.time()
inference_time_batch = (end_time - start_time)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step


In [11]:
print("\nAblated Model: No Dropout")
print(f"Accuracy: {test_accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"AUC: {test_auc:.4f}")
print(f"Parameters: {total_parameters}")
print(f"Test Loss: {test_loss:.4f}")
print(f"Inference Time (s/batch): {inference_time_batch:.4f}")


Ablated Model: No Dropout
Accuracy: 0.8720
Precision: 0.7436
Recall: 0.7250
F1-Score: 0.7342
AUC: 0.9049
Parameters: 11905.0
Test Loss: 0.4119
Inference Time (s/batch): 0.3991
